
# Классификация изображений

### Основная идея этого решения: взять предобученую на ImageNet сеть Xception и дообучить под нашу задачу. 

In [51]:
!nvidia-smi

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [53]:
!pip freeze > requirements.txt

# Основные настройки

In [54]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 15  # эпох на обучение
BATCH_SIZE           = 16 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
SEED                 = 2021

DATA_PATH = '../input/'
PATH = "../working/car/" # рабочая директория

In [55]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH,exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

### Аугментация данных

In [ ]:
## Install ImageDataAugmentor
!pip install --upgrade albumentations -q
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor -q

In [ ]:
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

In [ ]:
# Подключаем albumentations
AUGMENTATIONS = albumentations.Compose([
    albumentations.ShiftScaleRotate(
        p=0.8, #apply these to 80% of the images
        shift_limit=0.1, #translate by (-0.1%, 0.1%)
        scale_limit=0.1, #zoom by (-0.1%, 0.1%)
        rotate_limit=20, #rotate by (-20, 20)
        border_mode=cv2.BORDER_CONSTANT, value = [0,0,0], #fill emptyness with (0,0,0)
        ),
    albumentations.Blur(blur_limit=1, p=0.2),
    albumentations.ElasticTransform(alpha=0.1, sigma=5, alpha_affine=2,
                                     border_mode=cv2.BORDER_CONSTANT, value = [0,0,0], #fill emptyness with (0,0,0)
                                    ),
    albumentations.ToFloat(max_value=255),
])

### Генерация данных

In [ ]:
train_data_gen = ImageDataAugmentor(augment=AUGMENTATIONS, seed=SEED)
train_generator = train_data_gen.flow_from_directory(PATH+'train/', 
                                         batch_size=BATCH_SIZE, 
                                         shuffle=True,
                                         
                                         )

test_datagen = ImageDataAugmentor(augment=albumentations.ToFloat(max_value=255), seed=SEED)
test_generator = test_datagen.flow_from_directory(PATH+'train/', 
                                         batch_size=BATCH_SIZE, 
                                         shuffle=True,
                                         )

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    class_mode=None,
    shuffle=False,
    batch_size=BATCH_SIZE,)

# Построение модели

### Загружаем предобученную сеть:

In [ ]:
# base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)
base_model = tf.keras.applications.EfficientNetB7(weights='imagenet', include_top=False, input_shape = input_shape)

In [58]:
base_model.summary()

In [59]:
# Замораживаем обучение базовой модели
base_model.trainable = False

In [60]:
model = Sequential([ 
    base_model,
    GlobalMaxPool2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),
    Dense(CLASS_NUM, activation='softmax')  
])

model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [56]:
model.summary()

In [61]:
# Посмотрим на кол-во слоев и их обучаемость
print(len(model.layers))
len(model.trainable_variables)
for layer in model.layers:
    print(layer, layer.trainable)

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [62]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
# Добавим раннюю остановку с памятью в 5 итераций
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

Обучаем:

In [63]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [74]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [75]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [76]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# **Fine tuning**

In [71]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = int(len(base_model.layers)/2) # увеличим долю зафиксированных слоев до 1/2

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False
    
# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

In [72]:
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [73]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [81]:
base_model.trainable = True

In [82]:
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [83]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

# Предсказание на тестовых данных

In [77]:
test_sub_generator.samples

In [78]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [79]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('../working/submission.csv', index=False)

In [80]:
submission.head()

# Test-time augmentations (TTA)

In [ ]:
# Сделаем предсказания 10 раз
predictions = []
for _ in range(10):
  predictions.append(model.predict(test_sub_generator, verbose=1))
  test_sub_generator.reset()
predictions = np.array(predictions)
predictions.shape

final_predictions = predictions.mean(axis=0).argmax(axis=-1)

submission = pd.DataFrame({
    'Id': test_sub_generator.filenames,
    'Category': final_predictions
}, columns=['Id', 'Category'])


submission.to_csv('submission.csv', index=False)

# Результаты

* Для аугментации данных была использована библиотека albumentations.
* В качестве предобученной сети была выбрана EfficientNetB7.
* Был использован fine tuning с постепенным увеличинием количества обучаемых слоев в предобученной сети и применением весов из предыдущих итераций.
* Подбор параметров обучения (batch size, epochs, learning rate) дал результат 0.95745, что выше baseline.
* Попытка применить Test-time augmentations улучшения результата не дала.


